In [ ]:
import duckdb
import xml.etree.ElementTree as ET
from pathlib import Path
import pyarrow as pa

# Define the namespace
NS = {'marc': 'http://www.loc.gov/MARC21/slim'}

def stream_marc_records(file_path):
    """Stream MARC records to avoid loading entire file into memory"""
    context = ET.iterparse(file_path, events=('end',))
    for event, elem in context:
        if elem.tag.endswith('record'):
            # Extract relevant fields using proper namespace
            record_data = {
                'id': next((field.text for field in elem.findall('.//marc:controlfield[@tag="008"]', NS)), None),
                'autorstvo_kod': next((subfield.text 
                             for field in elem.findall('.//marc:datafield[@tag="100"]', NS)
                             for subfield in field.findall('.//marc:subfield[@code="7"]', NS)), None),
                'autorstvo': next((subfield.text 
                             for field in elem.findall('.//marc:datafield[@tag="100"]', NS)
                             for subfield in field.findall('.//marc:subfield[@code="a"]', NS)), None),
                'titul': next((subfield.text 
                             for field in elem.findall('.//marc:datafield[@tag="245"]', NS)
                             for subfield in field.findall('.//marc:subfield[@code="a"]', NS)), None),
                
                # Add more fields as needed
            }
            yield record_data
            elem.clear()

# Create DuckDB connection
con = duckdb.connect('data/skc2.db')

# Create table
con.execute("""
    CREATE TABLE IF NOT EXISTS marc_records (
        id VARCHAR,
        titul VARCHAR,
        autorstvo VARCHAR,
        autorstvo_kod VARCHAR
    )
""")

# Process in batches
batch_size = 10000
current_batch = []
pocitadlo = 0

for record in stream_marc_records('downloads/skc.xml'):
    current_batch.append(record)
    if len(current_batch) >= batch_size:
        pocitadlo += 1
        print(f"Ukládám batch {pocitadlo}.")
        # Convert batch to Arrow table and insert
        arrow_table = pa.Table.from_pylist(current_batch)
        con.execute("INSERT INTO marc_records (id, titul, autorstvo, autorstvo_kod) SELECT id, titul, autorstvo, autorstvo_kod FROM arrow_table")
        current_batch = []

# Insert remaining records
if current_batch:
    arrow_table = pa.Table.from_pylist(current_batch)
    con.execute("INSERT INTO marc_records SELECT * FROM arrow_table")


Ukládám batch 1.
Ukládám batch 2.
Ukládám batch 3.
Ukládám batch 4.
Ukládám batch 5.
Ukládám batch 6.
Ukládám batch 7.
Ukládám batch 8.
Ukládám batch 9.
Ukládám batch 10.
Ukládám batch 11.
Ukládám batch 12.
Ukládám batch 13.
Ukládám batch 14.
Ukládám batch 15.
Ukládám batch 16.
Ukládám batch 17.
Ukládám batch 18.
Ukládám batch 19.
Ukládám batch 20.
Ukládám batch 21.
Ukládám batch 22.
Ukládám batch 23.
Ukládám batch 24.
Ukládám batch 25.
Ukládám batch 26.
Ukládám batch 27.
Ukládám batch 28.
Ukládám batch 29.
Ukládám batch 30.
Ukládám batch 31.
Ukládám batch 32.
Ukládám batch 33.
Ukládám batch 34.
Ukládám batch 35.
Ukládám batch 36.
Ukládám batch 37.
Ukládám batch 38.
Ukládám batch 39.
Ukládám batch 40.
Ukládám batch 41.
Ukládám batch 42.
Ukládám batch 43.
Ukládám batch 44.
Ukládám batch 45.
Ukládám batch 46.
Ukládám batch 47.
Ukládám batch 48.
Ukládám batch 49.
Ukládám batch 50.
Ukládám batch 51.
Ukládám batch 52.
Ukládám batch 53.
Ukládám batch 54.
Ukládám batch 55.
Ukládám batch 56.
U